# Data Cleaning and Preprocessing for Vehicle Dataset

This notebook demonstrates the process of cleaning and preprocessing a vehicle dataset. We'll go through several steps including data loading, handling missing values, data type conversions, and group-based imputations.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn')
sns.set_palette("deep")

## 1. Data Loading

In [ ]:
# Load the data
file_path = r"C:\Users\90545\Desktop\Emir\Personal_Files\Data_Scraping_Projects\edmunds_project\scraping\edmunds_scraped_data_last.csv"
data = pd.read_csv(file_path, low_memory=False, index_col=0)
df = data.copy()

print(f"Dataset shape: {df.shape}")
df.head()

## 2. Initial Data Cleaning

In [ ]:
# Define columns to drop
columns_to_drop = ["vid", "vin", "listingUrl", "dealerInfo.address.street",
                   "dealerInfo.productFeatures.verified", "dealerInfo.address.stateName",
                   "stockNumber", "inTransit", "vehicleInfo.styleInfo.fuel.epaCombinedMPG",
                   "vehicleInfo.styleInfo.fuel.epaCityMPG", "vehicleInfo.styleInfo.fuel.epaHighwayMPG", "historyInfo.personalUseOnly"]

# Drop columns
df.drop(columns=columns_to_drop, inplace=True)

print(f"Dataset shape after dropping columns: {df.shape}")

## 3. Handling Missing Values and Special Cases

In [ ]:
# Replace '{}' with np.nan or special values
replacement_dict = {
    "prices.displayPrice": np.nan,
    "vehicleInfo.vehicleColors.exterior.genericName": np.nan,
    "vehicleInfo.vehicleColors.interior.genericName": np.nan,
    "historyInfo.historyProvider": "USER",
    "vehicleInfo.mileage": np.nan,
    "historyInfo.usageType": "Unknown_Usage_Type",
    "prices.baseMsrp": np.nan,
    "prices.totalMsrp": np.nan,
    "prices.loan.payment": np.nan,
    "historyInfo.ownerText": "Unknown",
    "sellersComments": np.nan
}

for col, value in replacement_dict.items():
    df[col] = df[col].replace("{}", value)

# Special case for engine type
df.loc[df["vehicleInfo.partsInfo.engineSize"].str.contains(r"\{\}") & 
       (df["vehicleInfo.partsInfo.fuelType"] == "{}") & 
       df["vehicleInfo.styleInfo.style"].str.contains(r'\d+cyl'), 
       "vehicleInfo.partsInfo.engineType"] = "gas"

print("Missing value handling complete.")

## 4. Data Type Conversions

In [ ]:
# Convert specific columns to float
float_columns = ["prices.displayPrice", "vehicleInfo.mileage", "prices.totalMsrp", "prices.baseMsrp", "prices.loan.payment"]
for col in float_columns:
    df[col] = df[col].astype("float")

print("Data type conversions complete.")
df.dtypes

## 5. Handling Electric Vehicles

In [ ]:
# Handle electric vehicles
columns_to_replace = ['vehicleInfo.partsInfo.engineSize', 'vehicleInfo.partsInfo.cylinders']
df[columns_to_replace] = df[columns_to_replace].replace('{}', np.nan)
df.loc[df['vehicleInfo.partsInfo.engineType'] == 'electric', columns_to_replace] = \
    df.loc[df['vehicleInfo.partsInfo.engineType'] == 'electric', columns_to_replace].fillna(0)

print("Electric vehicle handling complete.")

## 6. Group-based Imputation

In [ ]:
def process_column(df, column, group_cols):
    df[column] = df[column].replace("{}", np.nan)
    
    is_numeric = pd.api.types.is_numeric_dtype(df[column])
    
    if is_numeric:
        df[column] = df.groupby(group_cols)[column].transform(
            lambda x: x.fillna(x.median())
        )
        df[column] = df[column].fillna(df[column].median())
    else:
        df[column] = df.groupby(group_cols)[column].transform(
            lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan)
        )
        overall_mode = df[column].mode().iloc[0] if not df[column].mode().empty else "Unknown"
        df[column] = df[column].fillna(overall_mode)
    
    return df

group_cols = ['vehicleInfo.styleInfo.make', 'vehicleInfo.styleInfo.model', 'vehicleInfo.styleInfo.year']

columns_to_process = [
    'vehicleInfo.vehicleColors.exterior.genericName', 'vehicleInfo.vehicleColors.interior.genericName',
    'vehicleInfo.partsInfo.driveTrain', 'vehicleInfo.partsInfo.cylinders', 'vehicleInfo.partsInfo.engineSize',
    'vehicleInfo.partsInfo.engineType', 'vehicleInfo.partsInfo.fuelType', 'vehicleInfo.partsInfo.transmission',
    'vehicleInfo.styleInfo.trim', 'vehicleInfo.styleInfo.style', 'vehicleInfo.styleInfo.year',
    'vehicleInfo.styleInfo.bodyType', 'vehicleInfo.styleInfo.vehicleStyle', 'vehicleInfo.styleInfo.numberOfSeats',
    'historyInfo.ownerText', 'historyInfo.usageType', 'historyInfo.historyProvider', 'historyInfo.salvageHistory',
    'historyInfo.frameDamage', 'historyInfo.lemonHistory', 'historyInfo.theftHistory', 'historyInfo.accidentText',
    'computedDisplayInfo.priceValidation.dealType', 'prices.displayPrice', 'prices.loan.payment',
    'prices.baseMsrp', 'prices.totalMsrp', 'vehicleInfo.mileage'
]

for column in columns_to_process:
    df = process_column(df, column, group_cols)
    print(f"Processed {column}. NaN count: {df[column].isna().sum()}")

print("Group-based imputation complete.")

## 7. Final Data Overview

In [ ]:
print(f"Final dataset shape: {df.shape}")
print("\nMissing values summary:")
print(df.isna().sum())

# Display a heatmap of missing values
plt.figure(figsize=(12, 8))
sns.heatmap(df.isna(), yticklabels=False, cbar=False, cmap='viridis')
plt.title('Missing Value Heatmap')
plt.show()

## 8. Conclusion

We've successfully cleaned and preprocessed the vehicle dataset. The main steps included:
1. Loading the data
2. Initial data cleaning (dropping unnecessary columns)
3. Handling missing values and special cases
4. Data type conversions
5. Special handling for electric vehicles
6. Group-based imputation for remaining missing values

The dataset is now ready for further analysis or modeling tasks.